# <div style="text-align: center">18.335/6.337 Final Project - SOR</div>
### <div style="text-align: center">Created by Yusu Liu and Simon Batzner</div>

## <div style="text-align: center">Successive Over-Relaxation</div>

**to-do:**

- Compare vs. Julia's built-in SOR method
- Edge cases (SOR convergence is only provable for symmetric, positive definite matrices)
- Convergence plots (use residual error over starting residual, not in the code yet)
- Add explanation why we are slower than built-in functions (they call C code, optimized, ...)
- Build out error handling properly
- Memory savings possible?
- Convergence rate, proofs, etc. 
- Choice of omega (convergence comparisons, etc.)


**optional**:

- Add Hartree-Fock code
- Adaptive omega? 
- Parallel? 

In [222]:
function sor_self(A, b, ω, tol = 1e-10, maxiter = 100000)
    n = size(A,2)
    assert(size(A)==(n,n))
    
    ϕ = zeros(n)  # initial guess
    iter = 0 
    
    while (norm(b - A*ϕ, 2) > tol) && (iter <= maxiter)

        if iter == maxiter
            println("Maximum number of iterations reached: $(iter)")
            return ϕ
        end
        
        for i = 1:n
            
            σ = 0
            for j = 1:n
                if j != i
                   σ += A[i, j]*ϕ[j]  
                end
            end
            
            # ALTERNATIVE IMPLEMENTATION
            # ϕ[i] .= (1-ω)*ϕ[i] + ((ω/A[i,i]) * (b[i] - σ))
            
            # saves one multiplication 
            ϕ[i] .+= ω*((b[i] - σ)/A[i,i] - ϕ[i])                 

        end
        
        iter += 1
    end
    
    return ϕ, iter
end

sor_self (generic function with 3 methods)

In [223]:
function eval_err(n, ω, tol, num_sum = 1000)

    err = 0.0
    for j = 1:num_sum

        # random symmetric, positive definite matrix
        A = rand(Float64, n, n)
        A += A'
        A += n * eye(n)
        assert(issymmetric(A) && isposdef(A))
        b = rand(Float64, n)
        
        y = A\b
        ŷ, iter = sor_self(A, b, ω, tol)
        err += norm(ŷ - y) / norm(y)
    end

    err /= num_sum
    
    return err  # average error over num_sum iterations
end

eval_err (generic function with 2 methods)

In [224]:
function eval_t(n, ω, tol, num_sum = 1000)

    t_backslash = 0.0
    t_sor =  0.0
    
    for j = 1:num_sum
  
        A = rand(Float64, n, n)
        A += A'
        A += n * eye(n)
        assert(issymmetric(A) && isposdef(A))
        b = rand(Float64, n)
        
        t_backslash += @elapsed A\b
        t_sor += @elapsed sor_self(A, b, ω, tol)
    end

    t_backslash /= num_sum
    t_sor /= num_sum

    return t_backslash, t_sor # average time taken for A \ b, average time taken for SOR
end

eval_t (generic function with 2 methods)

### Measure the performance of SOR vs. Julia's built-in A \ b 

Solve several linear systems using Julia's built-in backslash operator and our implementation of SOR

Compare average time and error of the different implementations

In [235]:
n = 10

# create symmetric, positive definite matrix
A = rand(Float64, n, n)
A += A'
A += n * eye(n)
assert(issymmetric(A) && isposdef(A))

b = rand(Float64, n)
ω = 1.25
assert(0 < ω < 2)
tol = 1e-10
x = A \ b
ϕ, iter = sor_self(A, b, ω, tol)


println("\nA \\\ b ≈ SOR?  $(x ≈ ϕ)\n")
println("Converged in $(iter) iterations")

err = eval_err(n, ω, tol)

# Measure the performance of SOR vs. the backslash operator
t_backslash, t_sor = eval_t(n, ω, tol)
t_ratio = t_backslash / t_sor


# Print the measurement result.
println("\nComparison between A \\\ b and SOR")
println("------------------------------------")
println("A: n = $n, ω = $ω\n")

@printf "Average error in SOR vs. A \\\ b = %.10f%%\n\n" 100err

@printf "time for A \\\ b: \t%e seconds\n" t_backslash
@printf "time for SOR: \t\t%e seconds\n\n" t_sor

@printf "A \\\ b is %.5f times faster than SOR" 1/t_ratio


A \ b ≈ SOR?  true

Converged in 25 iterations

Comparison between A \ b and SOR
------------------------------------
A: n = 10, ω = 1.25

Average error in SOR vs. A \ b = 0.0000000047%

time for A \ b: 	2.578580e-06 seconds
time for SOR: 		2.162841e-04 seconds

A \ b is 83.87720 times faster than SOR

### Measure the performance of SOR vs. Julia's built-in SOR  

Solve several linear systems using Julia's built-in SOR method and our implementation of SOR

Compare average time and error of the different implementations